In [ ]:
from IPython.display import Image

## Implementación - Proyecto final

El propósito de este _notebook_ es presentar la implementación de la librería [ACO-TSP](https://github.com/C1587S/ACO-TSP/) utilizando un el conjunto de datos 150 de ciudades de China disponible en la base de datos _National Traveling Salesman Problems_ de la Universidad de Waterloo, disponible [aquí](https://www.math.uwaterloo.ca/tsp/world/countries.html). Esta implementación se realiza en forma de _pipeline_ para ser ejecutada en [`kubeflow`](https://www.kubeflow.org/) usando [`kale`](https://github.com/kubeflow-kale/kale). 

En particular, se siguen los siguientes pasos:


In [ ]:
Image("imgs/pipeline_kale.png")

In [1]:
import ant_colony as ac
import json
import timeit
import time
from multiprocessing import cpu_count

In [2]:
## parametros del algoritmo
data_path = 'datasets/ch71009.tsp'
init_node_ex = 0
hp_trials = 50
save_hp = True
seed = 19519159
n_cities = 150
n_cities_hp = 20

In [3]:
# lectura de datos 

## 200 ciudades - coordenadas y grafo
coord_df = ac.read_coord_data(data_path, n_cities=n_cities, seed=seed, coord_df=True)
G = ac.read_coord_data(data_path, n_cities=n_cities, seed=seed, coord_df=False)
## 30 ciudades - grafo hp
G_hp = ac.read_coord_data(data_path, n_cities=n_cities_hp, seed=seed, coord_df=False)

Problem with 71009 cities. Selected 30.
Problem with 71009 cities. Selected 30.
Problem with 71009 cities. Selected 10.


In [14]:
# visualizar y guardar mapa
map_nodes = ac.plot_nodes_map(coord_df, save=True, save_as='map_nodes')

In [15]:
# optimizacion de hiper-parametros y guardar en disco
study = ac.optim_h_params_mp(G_hp,
                          init_node = init_node_ex,
                          trials = hp_trials, 
                          save = save_hp)

[I 2021-05-19 14:00:20,062] A new study created in RDB with name: optimize_aco
[I 2021-05-19 14:00:21,129] Trial 0 finished with value: 2690.6691377268653 and parameters: {'n_ants': 20, 'max_iter': 100, 'rho': 0.3123127098659181, 'alpha': 2, 'beta': 1}. Best is trial 0 with value: 2690.6691377268653.
[I 2021-05-19 14:00:23,697] Trial 1 finished with value: 2462.6175124794086 and parameters: {'n_ants': 468, 'max_iter': 10, 'rho': 0.3830337251812589, 'alpha': 3, 'beta': 1}. Best is trial 1 with value: 2462.6175124794086.
[I 2021-05-19 14:00:23,798] Trial 2 finished with value: 5459.408053729692 and parameters: {'n_ants': 5, 'max_iter': 1, 'rho': 0.7258153206643667, 'alpha': 3, 'beta': 1}. Best is trial 1 with value: 2462.6175124794086.
[I 2021-05-19 14:00:23,869] Trial 3 finished with value: 6715.112156036722 and parameters: {'n_ants': 3, 'max_iter': 1, 'rho': 0.9750247872387847, 'alpha': 0, 'beta': 5}. Best is trial 1 with value: 2462.6175124794086.
[I 2021-05-19 14:00:24,636] Trial 4 f

Hyper-parameters saved in ./best_hiper_params.db


In [16]:
# cargar mejores hp
best_params = ac.load_params('best_hiper_params.db')

In [17]:
# instanciar colony_multiw con mejores hp
n_cpu = cpu_count()

colony_mw = ac.colony_multiw(G, 
                             init_node=0,  
                             n_workers = n_cpu,
                             verbose=True, k_verbose=10,
                              **best_params)

In [19]:
# solucionar el problema tsp con aco
start_time = time.time()
colony_mw.solve_tsp()
end_time = time.time()

secs = end_time-start_time
print("La solucion con pool tomó", secs, "segundos." )
print(f"Distancia {colony_mw.best_dist} kms.")

iter: 0 / 10 - dist: 215.69


------------------------------
Resumen:
	Nro. de hormigas: 703
	Iteraciones: 10
	Distancia: 214.62890760468554
	Nodo inicial: 0
	Ruta: [0, 25, 24, 27, 23, 7, 6, 3, 28, 18, 26, 2, 9, 15, 10, 13, 14, 5, 17, 16, 12, 19, 29, 21, 4, 20, 1, 22, 8, 11, 0]
------------------------------


In [21]:
# visualizar solución y guardar mapa como html
map_route = ac.plot_rout_map(coord_df, colony_mw.best_route, path_type='ants', 
                             save=True, save_as='map_aco_route')
map_route

In [22]:
# extraer mejor ruta
sln_r = colony_mw.best_route

In [23]:
# extraer mejor distancia
sln_d = colony_mw.best_dist

In [24]:
# diccionario con soluciones
sln_dic = {'route': sln_r, 'distance': float(sln_d)}

In [25]:
# guardar solución en disco
with open("aco_sln.json", "w") as outfile: 
    json.dump(sln_dic, outfile)
outfile.close()

## Comparación de tiempos con la antigua implementación

In [26]:
# instanciar colony_multiw con mejores hp
colony_old = ac.colony(G, 
                       init_node=0,  
                       verbose=True, k_verbose=10,
                       **best_params)

In [28]:
# solucionar el problema tsp con aco old
start_time = time.time()
colony_old.solve_tsp()
end_time = time.time()

secs = end_time-start_time
print("La solucion sin pool tomó", secs, "segundos." )
print(f"Distancia {colony_old.best_dist} kms.")

iter: 0 / 10 - dist: 196.66


------------------------------
Resumen:
	Nro. de hormigas: 703
	Iteraciones: 10
	Distancia: 196.65594273569621
	Nodo inicial: 0
	Ruta: [0, 15, 10, 27, 25, 24, 19, 20, 4, 17, 21, 3, 13, 9, 1, 22, 8, 5, 7, 14, 29, 2, 12, 16, 26, 23, 6, 11, 28, 18, 0]
------------------------------


## Referencias
- Kale: https://github.com/kubeflow-kale/kale
- Kubeflow: https://www.kubeflow.org/
- National Travelling Salesman problmes: https://www.math.uwaterloo.ca/tsp/world/countries.html
